In [14]:
import numpy as np
from collections import defaultdict
import os
from tqdm.auto import tqdm
import cv2
import torch
import json

from mmengine.config import Config
from mmengine.dataset import Compose

from mmseg.apis import init_model
import matplotlib.pyplot as plt

In [15]:
config_path = '/opt/ml/mmsegmentation/exp/upernet_02/upernet_02.py'
checkpoint_path = '/opt/ml/mmsegmentation/exp/upernet_02/iter_11200.pth'

In [16]:
# 테스트 데이터 경로 입력
IMAGE_ROOT = "/opt/ml/input/data/train/DCM/"
LABEL_ROOT = "/opt/ml/input/data/train/outputs_json"

CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}

cfg = Config.fromfile(config_path)
model = init_model(config_path, checkpoint_path, device='cuda:0')

def _prepare_data(imgs, model):
    for t in cfg.test_pipeline:
        if t.get('type') in ['LoadXRayAnnotations', 'TransposeAnnotations']:
            cfg.test_pipeline.remove(t)

    is_batch = True
    if not isinstance(imgs, (list, tuple)):
        imgs = [imgs]
        is_batch = False

    if isinstance(imgs[0], np.ndarray):
        cfg.test_pipeline[0]['type'] = 'LoadImageFromNDArray'

    # TODO: Consider using the singleton pattern to avoid building
    # a pipeline for each inference
    pipeline = Compose(cfg.test_pipeline)

    data = defaultdict(list)
    for img in imgs:
        if isinstance(img, np.ndarray):
            data_ = dict(img=img)
        else:
            data_ = dict(img_path=img)
        data_ = pipeline(data_)
        data['inputs'].append(data_['inputs'])
        data['data_samples'].append(data_['data_samples'])

    return data

/opt/ml/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '


Loads checkpoint by local backend from path: /opt/ml/mmsegmentation/exp/upernet_02/iter_11200.pth


In [17]:

# define colors
PALETTE = [
    (220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228),
    (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30),
    (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30), (165, 42, 42),
    (255, 77, 255), (0, 226, 252), (182, 182, 255), (0, 82, 0), (120, 166, 157),
    (110, 76, 0), (174, 57, 255), (199, 100, 0), (72, 0, 118), (255, 179, 240),
    (0, 125, 92), (209, 0, 151), (188, 208, 182), (0, 220, 176),
]
# 회색, 빨강, 민트
COM_PALETTE = [(188, 208, 182), (220, 20, 60), (0, 220, 176)]
def comp2rgb(label, classes=None):
    image_size = label.shape[1:] + (3, )
    image = np.zeros(image_size, dtype=np.uint8)
    
    if classes is None:
        print('None')
    else:
        for i, class_label in enumerate(label):
            if i in classes:
                image[class_label == 1] = COM_PALETTE[0]
                image[class_label == 2] = COM_PALETTE[1]
                image[class_label == 3] = COM_PALETTE[2]
    return image

# utility function
# this does not care overlap
def label2rgb(label, classes=None):
    image_size = label.shape[1:] + (3, )
    image = np.zeros(image_size, dtype=np.uint8)
    
    if classes is None:
        for i, class_label in enumerate(label):
            image[class_label == 1] = PALETTE[i]
    else:
        for i, class_label in enumerate(label):
            if i in classes:
                image[class_label == 1] = PALETTE[i]
    return image

def pixelwisecomparison(outputs, label):
    label_shape = (len(CLASSES), ) + tuple(outputs.shape[1:])
    comparison = np.zeros(label_shape, dtype=np.uint8)
    for idx, (class_output, class_label) in enumerate(zip(outputs, label)):
        comparison[idx][class_label == 1] = 1
        comparison[idx][class_output-class_label == 1] = 2
        comparison[idx][class_output-class_label == -1] = 3

    return comparison

filenames = ['ID004/image1661144691792.png', 'ID004/image1661144724044.png', 'ID009/image1661145407161.png', 'ID009/image1661145432967.png', 'ID014/image1661216876670.png', 'ID014/image1661216907060.png', 'ID019/image1661219523078.png', 'ID019/image1661219796151.png', 'ID024/image1661304293036.png', 'ID024/image1661304319731.png', 'ID029/image1661306136685.png', 'ID029/image1661306162532.png', 'ID034/image1661317748865.png', 'ID034/image1661317775801.png', 'ID039/image1661318938038.png', 'ID039/image1661318964936.png', 'ID054/image1661391074265.png', 'ID054/image1661391105683.png', 'ID060/image1661393384829.png', 'ID060/image1661393400879.png', 'ID064/image1661734980185.png', 'ID064/image1661735006875.png', 'ID070/image1661736017073.png', 'ID070/image1661736042863.png', 'ID075/image1661736870231.png', 'ID075/image1661736898823.png', 'ID080/image1661737362347.png', 'ID080/image1661737404974.png', 'ID085/image1661737831600.png', 'ID085/image1661737858331.png', 'ID090/image1661821775105.png', 'ID090/image1661821802814.png', 'ID095/image1661822623891.png', 'ID095/image1661822638602.png', 'ID100/image1661823638297.png', 'ID100/image1661823665932.png', 'ID105/image1661824735289.png', 'ID105/image1661824761643.png', 'ID111/image1661907834003.png', 'ID111/image1661907848327.png', 'ID115/image1661908159643.png', 'ID115/image1661908185790.png', 'ID120/image1661908761598.png', 'ID120/image1661908778667.png', 'ID125/image1661910199152.png', 'ID125/image1661910238659.png', 'ID130/image1662339673237.png', 'ID130/image1662339722889.png', 'ID136/image1662340433669.png', 'ID136/image1662340460163.png', 'ID278/image1664155311840.png', 'ID278/image1664155340191.png', 'ID283/image1664155969209.png', 'ID283/image1664155987422.png', 'ID288/image1664156956152.png', 'ID288/image1664156981600.png', 'ID293/image1664157316352.png', 'ID293/image1664157343885.png', 'ID298/image1664240709270.png', 'ID298/image1664240724225.png', 'ID303/image1664241119204.png', 'ID303/image1664241147172.png', 'ID308/image1664241429399.png', 'ID308/image1664241445936.png', 'ID313/image1664241736840.png', 'ID313/image1664241753946.png', 'ID318/image1664242930720.png', 'ID318/image1664242958139.png', 'ID323/image1664846188621.png', 'ID323/image1664846204441.png', 'ID328/image1664846838406.png', 'ID328/image1664846856735.png', 'ID333/image1664847413470.png', 'ID333/image1664847440074.png', 'ID338/image1664848589440.png', 'ID338/image1664848616528.png', 'ID343/image1664932424514.png', 'ID343/image1664932451633.png', 'ID348/image1664933179818.png', 'ID348/image1664933195576.png', 'ID353/image1664934044092.png', 'ID353/image1664934060702.png', 'ID358/image1664934876849.png', 'ID358/image1664934894024.png', 'ID363/image1664935962797.png', 'ID363/image1664935989808.png', 'ID368/image1665450162776.png', 'ID368/image1665450178483.png', 'ID373/image1665452035077.png', 'ID373/image1665452058937.png', 'ID378/image1665452927615.png', 'ID378/image1665452941530.png', 'ID383/image1665454683455.png', 'ID383/image1665454711973.png', 'ID388/image1665536805845.png', 'ID388/image1665536821260.png', 'ID393/image1665537862361.png', 'ID393/image1665537890819.png', 'ID399/image1665539161576.png', 'ID399/image1665539181057.png', 'ID403/image1665539937219.png', 'ID403/image1665539962902.png', 'ID408/image1665540854846.png', 'ID408/image1665540873549.png', 'ID413/image1666054988891.png', 'ID413/image1666055014770.png', 'ID418/image1666055793379.png', 'ID418/image1666055808396.png', 'ID423/image1666058234645.png', 'ID423/image1666058249428.png', 'ID428/image1666059689193.png', 'ID428/image1666059716951.png', 'ID433/image1666060113452.png', 'ID433/image1666060142188.png', 'ID438/image1666141346033.png', 'ID438/image1666141363652.png', 'ID443/image1666144094464.png', 'ID443/image1666144111171.png', 'ID448/image1666573448004.png', 'ID448/image1666573462779.png', 'ID453/image1666573743950.png', 'ID453/image1666573768043.png', 'ID458/image1666575044555.png', 'ID458/image1666575070798.png', 'ID463/image1666575511336.png', 'ID463/image1666575527139.png', 'ID468/image1666659863512.png', 'ID468/image1666659890125.png', 'ID473/image1666660367473.png', 'ID473/image1666660395613.png', 'ID478/image1666660861462.png', 'ID478/image1666660904025.png', 'ID483/image1666661332742.png', 'ID483/image1666661359745.png', 'ID488/image1666662049185.png', 'ID488/image1666662075807.png', 'ID493/image1666662711461.png', 'ID493/image1666662727834.png', 'ID499/image1666746427808.png', 'ID499/image1666746454260.png', 'ID503/image1666746789169.png', 'ID503/image1666746807797.png', 'ID508/image1666747732146.png', 'ID508/image1666747749143.png', 'ID513/image1666748180196.png', 'ID513/image1666748198922.png', 'ID518/image1666749056245.png', 'ID518/image1666749083578.png', 'ID523/image1667178735444.png', 'ID523/image1667178762956.png', 'ID528/image1667180274249.png', 'ID528/image1667180301380.png', 'ID533/image1667265171128.png', 'ID533/image1667265191721.png', 'ID538/image1667266190612.png', 'ID538/image1667266231591.png', 'ID543/image1667266674012.png', 'ID543/image1667266700981.png', 'ID548/image1667354140846.png', 'ID548/image1667354167046.png']
labelnames = ['ID004/image1661144691792.json', 'ID004/image1661144724044.json', 'ID009/image1661145407161.json', 'ID009/image1661145432967.json', 'ID014/image1661216876670.json', 'ID014/image1661216907060.json', 'ID019/image1661219523078.json', 'ID019/image1661219796151.json', 'ID024/image1661304293036.json', 'ID024/image1661304319731.json', 'ID029/image1661306136685.json', 'ID029/image1661306162532.json', 'ID034/image1661317748865.json', 'ID034/image1661317775801.json', 'ID039/image1661318938038.json', 'ID039/image1661318964936.json', 'ID054/image1661391074265.json', 'ID054/image1661391105683.json', 'ID060/image1661393384829.json', 'ID060/image1661393400879.json', 'ID064/image1661734980185.json', 'ID064/image1661735006875.json', 'ID070/image1661736017073.json', 'ID070/image1661736042863.json', 'ID075/image1661736870231.json', 'ID075/image1661736898823.json', 'ID080/image1661737362347.json', 'ID080/image1661737404974.json', 'ID085/image1661737831600.json', 'ID085/image1661737858331.json', 'ID090/image1661821775105.json', 'ID090/image1661821802814.json', 'ID095/image1661822623891.json', 'ID095/image1661822638602.json', 'ID100/image1661823638297.json', 'ID100/image1661823665932.json', 'ID105/image1661824735289.json', 'ID105/image1661824761643.json', 'ID111/image1661907834003.json', 'ID111/image1661907848327.json', 'ID115/image1661908159643.json', 'ID115/image1661908185790.json', 'ID120/image1661908761598.json', 'ID120/image1661908778667.json', 'ID125/image1661910199152.json', 'ID125/image1661910238659.json', 'ID130/image1662339673237.json', 'ID130/image1662339722889.json', 'ID136/image1662340433669.json', 'ID136/image1662340460163.json', 'ID278/image1664155311840.json', 'ID278/image1664155340191.json', 'ID283/image1664155969209.json', 'ID283/image1664155987422.json', 'ID288/image1664156956152.json', 'ID288/image1664156981600.json', 'ID293/image1664157316352.json', 'ID293/image1664157343885.json', 'ID298/image1664240709270.json', 'ID298/image1664240724225.json', 'ID303/image1664241119204.json', 'ID303/image1664241147172.json', 'ID308/image1664241429399.json', 'ID308/image1664241445936.json', 'ID313/image1664241736840.json', 'ID313/image1664241753946.json', 'ID318/image1664242930720.json', 'ID318/image1664242958139.json', 'ID323/image1664846188621.json', 'ID323/image1664846204441.json', 'ID328/image1664846838406.json', 'ID328/image1664846856735.json', 'ID333/image1664847413470.json', 'ID333/image1664847440074.json', 'ID338/image1664848589440.json', 'ID338/image1664848616528.json', 'ID343/image1664932424514.json', 'ID343/image1664932451633.json', 'ID348/image1664933179818.json', 'ID348/image1664933195576.json', 'ID353/image1664934044092.json', 'ID353/image1664934060702.json', 'ID358/image1664934876849.json', 'ID358/image1664934894024.json', 'ID363/image1664935962797.json', 'ID363/image1664935989808.json', 'ID368/image1665450162776.json', 'ID368/image1665450178483.json', 'ID373/image1665452035077.json', 'ID373/image1665452058937.json', 'ID378/image1665452927615.json', 'ID378/image1665452941530.json', 'ID383/image1665454683455.json', 'ID383/image1665454711973.json', 'ID388/image1665536805845.json', 'ID388/image1665536821260.json', 'ID393/image1665537862361.json', 'ID393/image1665537890819.json', 'ID399/image1665539161576.json', 'ID399/image1665539181057.json', 'ID403/image1665539937219.json', 'ID403/image1665539962902.json', 'ID408/image1665540854846.json', 'ID408/image1665540873549.json', 'ID413/image1666054988891.json', 'ID413/image1666055014770.json', 'ID418/image1666055793379.json', 'ID418/image1666055808396.json', 'ID423/image1666058234645.json', 'ID423/image1666058249428.json', 'ID428/image1666059689193.json', 'ID428/image1666059716951.json', 'ID433/image1666060113452.json', 'ID433/image1666060142188.json', 'ID438/image1666141346033.json', 'ID438/image1666141363652.json', 'ID443/image1666144094464.json', 'ID443/image1666144111171.json', 'ID448/image1666573448004.json', 'ID448/image1666573462779.json', 'ID453/image1666573743950.json', 'ID453/image1666573768043.json', 'ID458/image1666575044555.json', 'ID458/image1666575070798.json', 'ID463/image1666575511336.json', 'ID463/image1666575527139.json', 'ID468/image1666659863512.json', 'ID468/image1666659890125.json', 'ID473/image1666660367473.json', 'ID473/image1666660395613.json', 'ID478/image1666660861462.json', 'ID478/image1666660904025.json', 'ID483/image1666661332742.json', 'ID483/image1666661359745.json', 'ID488/image1666662049185.json', 'ID488/image1666662075807.json', 'ID493/image1666662711461.json', 'ID493/image1666662727834.json', 'ID499/image1666746427808.json', 'ID499/image1666746454260.json', 'ID503/image1666746789169.json', 'ID503/image1666746807797.json', 'ID508/image1666747732146.json', 'ID508/image1666747749143.json', 'ID513/image1666748180196.json', 'ID513/image1666748198922.json', 'ID518/image1666749056245.json', 'ID518/image1666749083578.json', 'ID523/image1667178735444.json', 'ID523/image1667178762956.json', 'ID528/image1667180274249.json', 'ID528/image1667180301380.json', 'ID533/image1667265171128.json', 'ID533/image1667265191721.json', 'ID538/image1667266190612.json', 'ID538/image1667266231591.json', 'ID543/image1667266674012.json', 'ID543/image1667266700981.json', 'ID548/image1667354140846.json', 'ID548/image1667354167046.json']


In [18]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true.reshape(29,-1)
    y_pred_f = y_pred.reshape(29,-1)
    intersection = np.sum(y_true_f * y_pred_f, -1)
    
    eps = 0.0001
    return (2. * intersection + eps) / (np.sum(y_true_f, -1) + np.sum(y_pred_f, -1) + eps)

In [19]:
def test(model, num):
    numlist = []
    with torch.no_grad():
        n_class = len(CLASSES)
        is_build = False
        for idx, (image_names, label_name) in tqdm(enumerate(zip(filenames, labelnames)), total=len(filenames)):
            image_path = os.path.join(IMAGE_ROOT, image_names)
            image = cv2.imread(image_path)

            data = _prepare_data(image, model)
            results = model.test_step(data)
            outputs = results[0].pred_sem_seg.data
            # outputs = results[0].seg_logits.data
            
            outputs = outputs.cpu().numpy()
            label_path = os.path.join(LABEL_ROOT, label_name)
            # outputs = (outputs > 0.65).to(outputs)
            # process a label of shape (H, W, NC)
            label_shape = tuple(image.shape[:2]) + (len(CLASSES), )
            label = np.zeros(label_shape, dtype=np.uint8)
            
            # read label file
            with open(label_path, "r") as f:
                annotations = json.load(f)
            annotations = annotations["annotations"]
            
            # iterate each class
            for ann in annotations:
                c = ann["label"]
                class_ind = CLASS2IND[c]
                points = np.array(ann["points"])
                
                # polygon to mask
                class_label = np.zeros(image.shape[:2], dtype=np.uint8)
                cv2.fillPoly(class_label, [points], 1)
                label[..., class_ind] = class_label
            label = label.transpose(2, 0, 1)
            dice = dice_coef(outputs, label)
            comparison = pixelwisecomparison(outputs, label)
            numlist.append([idx, dice[num], comparison])

               
    return sorted(numlist, key= lambda x: x[1])[:10]
    

In [20]:
class_val_lst = test(model, num=26)

100%|██████████| 160/160 [10:11<00:00,  3.82s/it]


In [21]:

for i, _, comparison in class_val_lst:
    label_path = os.path.join(LABEL_ROOT, labelnames[i])
    label_shape = tuple((2048, 2048)) + (len(CLASSES), )
    label = np.zeros(label_shape, dtype=np.uint8)
    
    # read label file
    with open(label_path, "r") as f:
        annotations = json.load(f)
    annotations = annotations["annotations"]
    
    # iterate each class
    for ann in annotations:
        c = ann["label"]
        class_ind = CLASS2IND[c]
        points = np.array(ann["points"])
        
        # polygon to mask
        class_label = np.zeros((2048, 2048), dtype=np.uint8)
        cv2.fillPoly(class_label, [points], 1)
        label[..., class_ind] = class_label
    label = label.transpose(2, 0, 1)
    image_path = os.path.join(IMAGE_ROOT, filenames[i])
    image = cv2.imread(image_path)
    print(labelnames[i])
    fig, ax = plt.subplots(1, 3, figsize=(36, 12))
    ax[0].imshow(image)
    ax[1].imshow(label2rgb(label))    # remove channel dimension
    ax[2].imshow(comp2rgb(comparison, [26]))

    plt.show()

ID029/image1661306136685.json
ID024/image1661304319731.json
ID533/image1667265191721.json
ID518/image1666749083578.json
ID075/image1661736898823.json
ID443/image1666144111171.json
ID438/image1666141346033.json
ID024/image1661304293036.json
ID039/image1661318964936.json
ID353/image1664934060702.json
